### Loading the data for Demo

## IMP LINK TO REFER 

https://keras-team.github.io/keras-tuner/documentation/hyperparameters/

https://keras-team.github.io/keras-tuner/documentation/tuners/

https://keras.io/api/layers/initializers/#randomnormal

In [1]:
from tensorflow.keras.datasets import boston_housing

(x_train, y_train), (x_test, y_test) = boston_housing.load_data()

c:\users\user\anaconda2\envs\mypersonal_mentor\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
c:\users\user\anaconda2\envs\mypersonal_mentor\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
c:\users\user\anaconda2\envs\mypersonal_mentor\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
# x_test.shape

In [3]:
#!pip install keras-tuner  # you just need for installation 

In [4]:
import tensorflow as tf  ##### TF > 2.0
import kerastuner as kt

print(tf.__version__)
print(kt.__version__)

2.3.0
1.0.2


In [5]:
from sklearn.preprocessing import StandardScaler  # Normalization 
from tensorflow.keras import models, layers  # Sequnetial , Sub Class API 
from kerastuner import HyperModel, RandomSearch, Hyperband, BayesianOptimization

from numpy.random import seed
seed(42)
import tensorflow
tensorflow.random.set_seed(42)

In [6]:
# models.save_mode

### Model without Hyperparameter Tuning

####  NN Model without HPT 

In [7]:
scaler = StandardScaler()
scaler.fit(x_train)  ## Doing fiting of scaleer on train

x_train_scaled = scaler.transform(x_train)  # using scaler which was fitted on train on train
x_test_scaled = scaler.transform(x_test)# using scaler which was fitted on train on test

In [8]:
# I have regression problem where which i need to predict House Price 

In [9]:
# import numpy as np
# np.random.randn(200)
#
#bias = zeros 

In [10]:
model = models.Sequential()  # Sequentail approach of developing my TF Model 

model.add(layers.Dense(8, activation='relu', input_shape=(x_train.shape[1],),
                       kernel_initializer='random_normal', bias_initializer='zeros'))


model.add(layers.Dense(16, activation='relu', kernel_initializer='random_normal',
                       bias_initializer='zeros'))


model.add(layers.Dropout(0.1)) ## Adding Drop Out Layer 


model.add(layers.Dense(1))  # Output layer with Neuron = 1 

model.compile(optimizer='rmsprop',loss='mse',metrics=['mse'])

In [11]:
history = model.fit(x_train_scaled, y_train, validation_split=0.2, epochs=15)

Epoch 1/15
11/11 [==============================] - 0s 12ms/step - loss: 570.3118 - mse: 570.3118 - val_loss: 639.6743 - val_mse: 639.6743
Epoch 2/15
11/11 [==============================] - 0s 2ms/step - loss: 566.9042 - mse: 566.9042 - val_loss: 635.9645 - val_mse: 635.9645
Epoch 3/15
11/11 [==============================] - 0s 3ms/step - loss: 563.0008 - mse: 563.0008 - val_loss: 631.5813 - val_mse: 631.5813
Epoch 4/15
11/11 [==============================] - 0s 3ms/step - loss: 558.3314 - mse: 558.3314 - val_loss: 625.6187 - val_mse: 625.6187
Epoch 5/15
11/11 [==============================] - 0s 3ms/step - loss: 552.7308 - mse: 552.7308 - val_loss: 619.0966 - val_mse: 619.0966
Epoch 6/15
11/11 [==============================] - 0s 3ms/step - loss: 545.7599 - mse: 545.7599 - val_loss: 611.0032 - val_mse: 611.0032
Epoch 7/15
11/11 [==============================] - 0s 3ms/step - loss: 537.2849 - mse: 537.2849 - val_loss: 601.7495 - val_mse: 601.7495
Epoch 8/15
11/11 [===============

In [12]:
mse = model.evaluate(x_test_scaled, y_test)[1]

4/4 [==============================] - 0s 647us/step - loss: 458.2523 - mse: 458.2523


In [13]:
print('MSE without tuning: {}'.format(mse))

MSE without tuning: 458.2522888183594


### Building the Hypermodel

In [19]:

class RegressionHyperModel(HyperModel):
    def __init__(self, input_shape):
        self.input_shape = input_shape

    def build(self, hp):
        model = models.Sequential()
        model.add(
            layers.Dense(
                units=  hp.Int('units', 8, 64, 4, default=8),   # [8,12,16,20,24,28,32,36,40,44,48,52,56,62]
                activation=hp.Choice(
                    'dense_activation',
                    values=['relu', 'tanh', 'sigmoid'],
                    default='relu'),
                input_shape=input_shape,
                kernel_initializer='random_normal', bias_initializer='zeros'   #hp.Choice is used to define a categorical hyperparameter such as the activation function. The search space below, named “dense_activation”, will choose between “relu”, “tanh”, and “sigmoid” functions, with a default value set to “relu”
            )
        )
        
        model.add(
            layers.Dense(
                units=hp.Int('units', 16, 64, 4, default=16), #######  [ 16,20,24,28,32,36,40,44,48,52,56,60,64]
                activation=hp.Choice(
                    'dense_activation',
                    values=['relu', 'tanh', 'sigmoid'],
                    default='relu'),
                kernel_initializer='random_normal', bias_initializer='zeros'   # Both hp.Int and hp.Float requires a name, minimum value and maximum value, while the step size and default value is optional
            )
        )
        
        model.add(
            layers.Dropout(
                hp.Float(
                    'dropout',
                    min_value=0.0,
                    max_value=0.1,     # [0.0,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1]
                    default=0.005,
                    step=0.01)
            )
        )
        
        model.add(layers.Dense(1, kernel_initializer='random_normal', bias_initializer='zeros'))
        
        model.compile(
            optimizer='rmsprop',loss='mse',metrics=['mse']
        )
        
        return model
    
    


### Initialize hypermodel

In [20]:
input_shape = (x_train.shape[1],)
hypermodel = RegressionHyperModel(input_shape)

In [22]:
# from sklearn.model_selection import GridSearchCV

# import xgboost 

# xgb = xgboost(n_jobs = -1 )

# GridSearchCV(xgb,)

### Build Random Search Tuner

In [21]:
import os

In [27]:
tuner_rs = RandomSearch(
            hypermodel,
            objective='mse',
            seed=42,
            max_trials=20,
            executions_per_trial=2, overwrite=True,directory=os.path.normpath('C:\\Arihant\\Personal\\Mentorship\\UpGrad\\KerasTuner_Random')
        )




#max_trials represents the number of hyperparameter combinations that will be tested by the tuner, while execution_per_trial is the number of models that should be built and fit for each trial for robustness purposes.

### Run Random Search

In [28]:
tuner_rs.search(x_train_scaled, y_train, epochs=10, validation_split=0.2, verbose=0)  

## Main code which whill trigger your HPT 

INFO:tensorflow:Oracle triggered exit


### Evaluate Random Search

In [29]:
best_model = tuner_rs.get_best_models(num_models=1)[0]
mse_rs = best_model.evaluate(x_test_scaled, y_test)[1]

4/4 [==============================] - 0s 892us/step - loss: 94.0469 - mse: 94.0469


In [33]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8)                 112       
_________________________________________________________________
dense_1 (Dense)              (None, 16)                144       
_________________________________________________________________
dropout (Dropout)            (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 17        
Total params: 273
Trainable params: 273
Non-trainable params: 0
_________________________________________________________________


In [32]:
best_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 60)                840       
_________________________________________________________________
dense_1 (Dense)              (None, 60)                3660      
_________________________________________________________________
dropout (Dropout)            (None, 60)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 61        
Total params: 4,561
Trainable params: 4,561
Non-trainable params: 0
_________________________________________________________________


In [30]:
print('Random search MSE: ', mse_rs)

Random search MSE:  94.0468978881836


### Build, Run and Evaluate Hyperband Tuner

In [84]:
tuner_hb = Hyperband(
            hypermodel,
            max_epochs=5,
            objective='mse',
            seed=42,
            executions_per_trial=2 ,directory=os.path.normpath('C:\\Arihant\\Personal\\Mentorship\\UpGrad\\KerasTuner_Hyperband')
       )

tuner_hb.search(x_train_scaled, y_train, epochs=10, validation_split=0.2, verbose=0)

best_model = tuner_hb.get_best_models(num_models=1)[0]
mse_hb = best_model.evaluate(x_test_scaled, y_test)[1]

INFO:tensorflow:Reloading Oracle from existing project C:\Arihant\Personal\Mentorship\UpGrad\KerasTuner_Hyperband\untitled_project\oracle.json
INFO:tensorflow:Reloading Tuner from C:\Arihant\Personal\Mentorship\UpGrad\KerasTuner_Hyperband\untitled_project\tuner0.json
INFO:tensorflow:Oracle triggered exit
4/4 [==============================] - 0s 707us/step - loss: 589.9302 - mse: 589.9302


In [85]:
print('Hyperband MSE: ', mse_hb)

Hyperband MSE:  589.93017578125


### Build, Run and Evaluate Bayesian Optimization Tuner

In [86]:
tuner_bo = BayesianOptimization(
            hypermodel,
            objective='mse',
            max_trials=10,
            seed=42,
            executions_per_trial=2,
            directory=os.path.normpath('C:\\Arihant\\Personal\\Mentorship\\UpGrad\\KerasTuner_Bayesian')
        
        )

tuner_bo.search(x_train_scaled, y_train, epochs=10, validation_split=0.2, verbose=0)

best_model = tuner_bo.get_best_models(num_models=1)[0]
mse_bo = best_model.evaluate(x_test_scaled, y_test)[1]

INFO:tensorflow:Reloading Oracle from existing project C:\Arihant\Personal\Mentorship\UpGrad\KerasTuner_Bayesian\untitled_project\oracle.json
INFO:tensorflow:Reloading Tuner from C:\Arihant\Personal\Mentorship\UpGrad\KerasTuner_Bayesian\untitled_project\tuner0.json
INFO:tensorflow:Oracle triggered exit
4/4 [==============================] - 0s 771us/step - loss: 458.7108 - mse: 458.7108


In [88]:
print('Bayesian Optimization MSE: ', mse_bo)

Bayesian Optimization MSE:  458.7108154296875
